<a href="https://colab.research.google.com/github/shelke16/HPC/blob/main/addition_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y nvidia-cuda-toolkit

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,700 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubunt

In [2]:
!apt-get install -y build-essential
!apt-get install -y nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nvidia-cuda-toolkit is already the newest version (11.5.1-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 8.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660425 sha256=39dbdb64e5f5c9b7c61a3a2014a3eb2341cb8058f70b3ff1fe3c2cd293377206
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [6]:
!apt-get update
!apt-get install -y nvidia-cuda-toolkit
!apt-get install -y build-essential
!pip install pycuda --upgrade
!apt install nvidia-driver-utils

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 128 kB in 1s (93.3 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Don

In [8]:
!apt-get update
!apt-get install -y build-essential
!apt-get install -y nvidia-cuda-toolkit

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [9]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from pycuda.compiler import SourceModule
import time

# CUDA Kernel code for vector addition
kernel_code = """
__global__ void add(int *arr1, int *arr2, int *arr3, int size) {
    int block_id = blockIdx.x * blockDim.x + threadIdx.x;
    if(block_id < size) {
        arr3[block_id] = arr1[block_id] + arr2[block_id];
    }
}
"""

# Function to fill arrays with random values
def fill_array(arr, size):
    for i in range(size):
        arr[i] = np.random.randint(0, 100)

# Function to add arrays on the CPU
def add_cpu(arr1, arr2, result, size):
    for i in range(size):
        result[i] = arr1[i] + arr2[i]

# Function to print arrays
def print_matrix(arr, size):
    for i in range(size):
        print(arr[i], end=" ")
    print()

# Main code to run CUDA vector addition
size = int(input("Enter size of vector: "))

# Allocate memory for CPU arrays
arr1_cpu = np.zeros(size, dtype=np.int32)
arr2_cpu = np.zeros(size, dtype=np.int32)
result_cpu = np.zeros(size, dtype=np.int32)

# Fill arrays with random values
fill_array(arr1_cpu, size)
print("Array 1:")
print_matrix(arr1_cpu, size)
fill_array(arr2_cpu, size)
print("Array 2:")
print_matrix(arr2_cpu, size)

# Allocate memory for GPU arrays
arr1_gpu = cuda.mem_alloc(arr1_cpu.nbytes)
arr2_gpu = cuda.mem_alloc(arr2_cpu.nbytes)
result_gpu = cuda.mem_alloc(result_cpu.nbytes)

# Copy data from CPU to GPU
cuda.memcpy_htod(arr1_gpu, arr1_cpu)
cuda.memcpy_htod(arr2_gpu, arr2_cpu)

# Compile the CUDA code
mod = SourceModule(kernel_code)
add_kernel = mod.get_function("add")

# Block and grid sizes
block_size = 256
grid_size = (size + block_size - 1) // block_size

# Timing the kernel execution
start_time = time.time()

# Launch kernel
add_kernel(arr1_gpu, arr2_gpu, result_gpu, np.int32(size), block=(block_size, 1, 1), grid=(grid_size, 1))

# Copy result from GPU to CPU
cuda.memcpy_dtoh(result_cpu, result_gpu)

# End timing
end_time = time.time()

# Print result from GPU
print("GPU result:")
print_matrix(result_cpu, size)

# Calculate elapsed time
print("Elapsed Time for GPU = {:.4f} seconds".format(end_time - start_time))

# CPU addition for comparison
start_time_cpu = time.time()
add_cpu(arr1_cpu, arr2_cpu, result_cpu, size)
end_time_cpu = time.time()

# Print result from CPU
print("CPU result:")
print_matrix(result_cpu, size)

# Calculate elapsed time for CPU
print("Elapsed Time for CPU = {:.4f} seconds".format(end_time_cpu - start_time_cpu))


RuntimeError: cuInit failed: no CUDA-capable device is detected

In [10]:
%%writefile vector_add.cu

#include<iostream>
#include<bits/stdc++.h>
#include<cuda.h>
#define BLOCK_SIZE 16
using namespace std;
void fill_array(int *arr,int size){
for(int i = 0;i < size; i++){
arr[i] = rand() % 100;
}
}
void add_cpu(int *arr1, int *arr2, int *result, int size){
for(int i = 0;i < size; i++){
result[i] = arr1[i] + arr2[i];
}
}
void print_matrix(int *arr, int size){
for(int i = 0; i < size; i++){
cout << arr[i] << " ";
}
cout << endl;
}
__global__ void add(int *arr1, int *arr2, int *arr3,int size){
int block_id = blockIdx.x * blockDim.x + threadIdx.x;
if(block_id < size){
arr3[block_id] = arr1[block_id] + arr2[block_id];
}
}
int main(){
int *arr1_cpu,*arr2_cpu,*result_cpu;
int size;
cout << "Enter size of vector: ";
cin >> size;
arr1_cpu = new int[size];
arr2_cpu = new int[size];
result_cpu = new int[size];
fill_array(arr1_cpu,size);
cout << "Array 1: ";
print_matrix(arr1_cpu,size);
fill_array(arr2_cpu,size);
cout << "Array 2: ";
print_matrix(arr2_cpu,size);
int *arr1_gpu,*arr2_gpu,*result_gpu;
cudaMallocManaged(&arr1_gpu, size * sizeof(int));
cudaMallocManaged(&arr2_gpu, size * sizeof(int));
cudaMallocManaged(&result_gpu, size * sizeof(int));
cudaMemcpy(arr1_gpu,arr1_cpu,size * sizeof(int),cudaMemcpyHostToDevice);
cudaMemcpy(arr2_gpu,arr2_cpu,size * sizeof(int),cudaMemcpyHostToDevice);
cudaEvent_t start,stop;
float elapsedTime;
dim3 dimGrid(size + BLOCK_SIZE - 1 / BLOCK_SIZE);
dim3 dimBlock(BLOCK_SIZE);
cudaEventCreate(&start);
cudaEventCreate(&stop);
cudaEventRecord(start,0);
add<<<dimGrid,dimBlock>>>(arr1_gpu,arr2_gpu,result_gpu,size);
cudaEventRecord(stop,0);
cudaEventSynchronize(stop);
cudaEventElapsedTime(&elapsedTime,start,stop);
cudaEventDestroy(start);
cudaEventDestroy(stop);
cudaMemcpy(result_cpu,result_gpu,size * sizeof(int),cudaMemcpyDeviceToHost);
cout << "GPU result:\n";
print_matrix(result_cpu,size);
cout<<"Elapsed Time = "<<elapsedTime<<" milliseconds" << endl;
cudaFree(arr1_gpu);
cudaFree(arr2_gpu);
cudaFree(result_gpu);
cudaEventCreate(&start);
cudaEventCreate(&stop);
cudaEventRecord(start,0);
add_cpu(arr1_cpu,arr2_cpu,result_cpu,size);
cudaEventRecord(stop,0);
cudaEventSynchronize(stop);
cudaEventElapsedTime(&elapsedTime,start,stop);
cudaEventDestroy(start);
cudaEventDestroy(stop);
cout << "CPU result:\n";
print_matrix(result_cpu,size);
cout<<"Elapsed Time = "<<elapsedTime<<" milliseconds" << endl;
return 0;
}

Writing vector_add.cu


In [11]:
!nvcc vector_add.cu -o vector_add

In [12]:
!./vector_add

Enter size of vector: 4
Array 1: 83 86 77 15 
Array 2: 93 35 86 92 
GPU result:
0 0 0 0 
Elapsed Time = 0 milliseconds
CPU result:
176 121 163 107 
Elapsed Time = 0 milliseconds
